# Simulate STREAMS for Point Source for Database at IRAP
This notebook simulates one source at 10mCrab in focus with a duration of 30.1s over the 50 brightest pixels (see XSPEC model 10mCrab.xcm)

Source spectral model:   
```
method leven 10 0.01    
abund wilm     
xsect vern     
statistic cstat     
cosmo 70  0    0.7      
xset delta 0.01    
systematic 0
model TBabs*powerlaw
         0.4       0.004           0           0      100000       1e+06
         2.1       0.021          -3          -2           9          10
      9.5e-2      9.5e-4           0           0       1e+20       1e+24
```

Simulation steps   
1. Read simulation parameters and derived parameters   
2. HEASOFT `xspec`: create xspec model file    
3. SIXTE `simputfile`: Create simput file with photons distribution    
4. SIXTE `sixtesim`: Run simulation to get   
    4.1 ImpactList - piximpact file for ALL photons    
    4.2 EventList - which photons (PH_ID) impact in each pixel of the detector (possibly including background and XTalk)   
    4.3 PixImpactList: piximpact file for each pixel with impacts (PH_ID) (needed by xifusim)   - 50 brightest pixels
5. Get list of pixels w/ impacts.   
    5.1. `xifusim`: Do single-pixel (NOMUX) xifusim simulation     
    
    

## Import routines and read parameters

In [ ]:
import os
from subprocess import run
import tempfile
import glob
from astropy.io import fits
from astropy.table import Table
import numpy as np
from xspec import Xset, Model, AllModels
import auxiliary as aux

In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = f"{tmpDir}:{os.environ['PFILES']}"
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]

In [ ]:
def vprint(*args, **kwargs):
    """
    Print function that can be turned on/off with the verbose variable.
    """
    if verbose > 0:
        print(*args, **kwargs)


### Read simulation parameters   

In [ ]:
def get_parameters():
    """
    Get parameters for pairs detection analysis.
    If running in a Jupyter Notebook, use default parameters.
    If running as a script (e.g., SLURM), parse command line arguments.
    """
    #flux_mcrab = 950 #mCrab -->most similar case to input file
    # 1 mCrab = 90 counts/s in the 2-10 keV band and 2.0533E-11 erg/cm^2/s
    if aux.is_notebook():
        # Default parameters for interactive use
        print("Running in notebook mode for source simulation")
        return {
            "model_xcm": "10mCrab.xcm",
            "flux_mcrab": 10.,
            "exposure": 30.1, #s
            "nonxbgd": "no", #yes/no
            "XTalk": "none", # all, elec, therm, tdm_prop, tdm_prop1, tdm_prop2, tdm_der, none
            "filter": "nofilt",
            "focus": "infoc",
            "xifusim_xml_version": "v5_20250621",
            "nbrightest": 50,
            "verbose": 1
        }
    else:
        import argparse
        parser = argparse.ArgumentParser(description="Simulate Point Source DB spectrum with SIXTE & XIFUSIM")
        parser.add_argument("--model_xcm", type=str, help="XCM model file for the source")
        parser.add_argument("--flux_mcrab", type=float, required=True, help="Source flux in mCrab")
        parser.add_argument("--exposure", type=float, required=True, help="Exposure time in seconds")
        parser.add_argument("--nonxbgd", type=str, choices=["yes", "no"], default="no", help="Include non-X-ray background (yes/no)")
        parser.add_argument("--XTalk", type=str, choices=["all", "elec", "therm", "tdm_prop", "tdm_prop1", "tdm_prop2", "tdm_der", "none"], 
                            default="none", help="Crosstalk model to use")
        parser.add_argument("--filter", type=str, default="nofilt", help="Filter to use")
        parser.add_argument("--focus", type=str, default="infoc", help="Focus setting to use")
        parser.add_argument("--xifusim_xml_version", type=str, required=True, help="Version of the XIFU simulator XML files",
                            choices=["v5_20250621", "v3_20240215"])
        parser.add_argument("--nbrightest", type=int, default=50, help="Number of brightest pixels to consider")
        parser.add_argument("--verbose", type=int, default=0, help="Verbosity level")
        args = parser.parse_args()
        return vars(args)

In [ ]:
params = get_parameters()
model_xcm = params["model_xcm"]
flux_mcrab = params["flux_mcrab"]
exposure = params["exposure"]
nonxbgd = params["nonxbgd"]
XTalk = params["XTalk"]
filter = params["filter"]
focus = params["focus"]
xifusim_xml_version = params["xifusim_xml_version"]
nbrightest_pixels = params["nbrightest"]
aux.verbose = params["verbose"]
print(params)

In [ ]:
# get model name from xcm file
model = os.path.splitext(os.path.basename(model_xcm))[0]
RA=0.
Dec=0.
sampling_rate=130210 #Hz
prebuff_xifusim=1500  #prebuffer samples for xifusim
Emin=2.0
Emax=10.0
flux = flux_mcrab * 2.0533E-11
#rate = flux_mcrab * 90 #counts/s
flux_mcrab_str = f"{flux_mcrab:.2f}".replace('.', 'p')
exposure_str = f"{exposure:.1f}".replace('.', 'p')

### Set XML files for simulations (sixtesim and xifusim)

In [ ]:
## Set XML file for sixte simulation
xmldir = f"{SIXTE}/share/sixte/instruments/new-athena-xifu/internal_design_goal"
xml_sixtesim = f"{xmldir}/xifu_{filter}_defoc.xml"
vprint(f"Using sixtesim XML file: {xml_sixtesim}")

# Find name of (unique) xml file in indir directory
xml_xifusim = glob.glob(f"./{xifusim_xml_version}/config*.xml")
if len(xml_xifusim) != 1:
    raise FileNotFoundError(f"Error: expected 1 XML file but found {len(xml_xifusim)}")
xml_xifusim = xml_xifusim[0]
vprint(f"Using XIFUSIM XML file: {xml_xifusim}")

In [ ]:
# set string to name files based on input parameters
filestring_simput = f"./{xifusim_xml_version}/{model}_flux{flux_mcrab_str}mcrab"
filestring = f"./{xifusim_xml_version}/{model}_flux{flux_mcrab_str}mcrab_{filter}_{focus}"
print(filestring)

## Do SIXTESIM simulation    
Files required:   
- XSPEC file 
- SIMPUT file   

### Create XSPEC model file   

In [ ]:
# is spectral model file does not exist, create it

if not os.path.exists(model_xcm):
    # Clear all models
    AllModels.clear()
    # define XSPEC parameters
    Xset.abund = "wilm"
    Xset.cosmo = "70 0. 0.7"
    Xset.xsect = "vern"
    mcmod = Model("TBabs*powerlaw")
    mcmod.TBabs.nH = 0.4
    mcmod.powerlaw.PhoIndex = 2.1
    mcmod.powerlaw.norm = 9.5E-2 * flux_mcrab / 10.0  # scale norm to desired flux
    #retrieve the flux value
    AllModels.calcFlux(f"{Emin} {Emax}")
    model_flux = AllModels(1).flux[0]
    # Save the model to the specified .xcm file path
    Xset.save(xcm)
    vprint(f"Model saved to {xcm}")
    mcmod.show()
else:
    print(f"Using existing model file: {model_xcm}")

### Create simput file

In [ ]:
# run simputfile to create the simput file
simputfile = f"{filestring_simput}_simput.fits"
if not os.path.exists(simputfile):
        comm = (f'simputfile Simput={simputfile} RA={RA} Dec={Dec} '
                f'srcFlux={flux} Emin={Emin} Emax={Emax} '
                f'XSPECFile={model_xcm} clobber=yes')
        vprint(f"Running {comm}")
        # Run the command through the subprocess module
        output_simputfile = run(comm, shell=True, capture_output=True)
        #print(output_simputfile.stdout.decode())
        assert output_simputfile.returncode == 0, f"simputfile failed to run: {comm}"
        assert os.path.exists(simputfile), f"simputfile did not produce an output file"

### Run SIXTESIM and get PIXIMPACT file 

In [ ]:
evtfile = f"{filestring}_evt.fits"
photfile = f"{filestring}_photon.fits"
impfile = f"{filestring}_impact.fits"
if not os.path.exists(evtfile) or not os.path.exists(photfile) or not os.path.exists(impfile):    
        comm = (f'sixtesim PhotonList={photfile} Simput={simputfile} '
                f'ImpactList={impfile} EvtFile={evtfile} doCrossTalk={XTalk} '
                f'XMLFile={xml_sixtesim} Background={nonxbgd} RA={RA} Dec={Dec} ' 
                f'Exposure={exposure} clobber=yes')
        vprint(comm)
        output_sixtesim = run(comm, shell=True, capture_output=True)
        assert output_sixtesim.returncode == 0, f"sixtesim failed to run"
        assert os.path.exists(evtfile), f"sixtesim did not produce an output file"
        assert os.path.exists(photfile), f"sixtesim did not produce an output file"
        assert os.path.exists(impfile), f"sixtesim did not produce an output file"

### Get list of pixels with counts produced by sixtesim 

In [ ]:
#read column PIXID from evtfile and save to a list of unique pixels
hdulist = fits.open(evtfile, mode='update')
evtdata = hdulist[1].data.copy()
pixels_with_impacts = np.unique(evtdata["PIXID"])
vprint(f"Number of pixels with impacts: {len(pixels_with_impacts)}")
hdulist.close()

# get pixels used and the events in each pixel
nimpacts_inpix = dict()
phid_impacts_inpix = dict()
for pixel in pixels_with_impacts:
    phid_impacts_inpix[pixel] = evtdata['PH_ID'][evtdata['PIXID'] == pixel]
    nimpacts_inpix[pixel] = len(phid_impacts_inpix[pixel])

#print number of impacts per pixel sorted by number of impacts
sim_brightest_pixels = sorted(nimpacts_inpix, key=nimpacts_inpix.get, reverse=True)
for pixel in sim_brightest_pixels:
    vprint(f"Pixel {pixel}: {nimpacts_inpix[pixel]} impacts")


#print the PH_ID of impacts in pixels
for key, value in phid_impacts_inpix.items():
    vprint(f"Pixel {key}: ")
    vprint(f"      PH_ID:{value}")

In [ ]:
# get impacts in the 'nbrightest_pixels' simulated pixels

for ipix in range(nbrightest_pixels):
    pixel = sim_brightest_pixels[ipix]
    nimpacts_sim = nimpacts_inpix[pixel]


### Read sixtesim output data

In [ ]:
# open sixtesim ImpactList and read data 
hdulist = fits.open(impfile)
impdata = hdulist[1].data.copy()
hdulist.close()
#vprint(impdata)

## Do XIFUSIM simulation  for the simulated brightest pixels

In [ ]:
vprint(f"Simulated Brightest pixels: {sim_brightest_pixels}")

### Extract a piximpact file for each interesting pixel   
- create a subsample of the piximpact file selecting only those rows where PH_ID is in the list of the impacts in the pixel   
- copy src impacts from impact file and bkgs from event file (background are not in the impact list, only in the event file)

In [ ]:
for ipix in range(nbrightest_pixels):
    ipixel = sim_brightest_pixels[ipix]
    vprint(f"Checking existence of piximpact file for pixel {ipixel}")
    # create a subsample of the piximpact file selecting only those rows where PH_ID is in 
    # the list of the impacts in the pixel
    piximpactfile = f"{filestring}_pixel{ipixel}_piximpact.fits"
    # if file does not exist, create it
    if not os.path.exists(piximpactfile):
        # create a mask to select only the rows with the impacts in this pixel
        mask = np.isin(impdata['PH_ID'], phid_impacts_inpix[ipixel])
        # create a new table with the selected rows
        newtable = Table(impdata[mask])
        # sort newtable according to TIME
        newtable.sort('TIME')
            
        # add new columns X,Y,U,V, GRADE1, GRADE2, TOTALEN with the value 0 and PIXID with the value of ipixel
        newtable['X'] = 0.
        newtable['Y'] = 0.
        newtable['U'] = 0.
        newtable['V'] = 0.
        newtable['GRADE1'] = 0
        newtable['GRADE2'] = 0
        newtable['TOTALEN'] = 0
        newtable['PIXID'] = 1 #requirement for xifusim

        # name the new table 'PIXELIMPACT'
        newtable.meta['EXTNAME'] = 'PIXELIMPACT'
    
        # write the new table to a new FITS file
        newtable.write(piximpactfile, format='fits', overwrite=True)

        # print the name of the new file rewriting the output line
        vprint(f"Created {piximpactfile} for pixel {ipixel}")


### Run the xifusim simulation (with XML for single pixel)   
    - simulate time between min and max TIME in piximpact   
    - xifusim simulation    
    - re-establish correct PIXID    

In [ ]:
#for each piximpact file, run xifusim
prebuffer = 1500
phsims_inpix = dict()
nphsims_inpix = dict()
skipped_photons_inpix = dict()
skipped_xifusim = []   

for ipix in range(nbrightest_pixels):
    ipixel = sim_brightest_pixels[ipix]
    piximpactfile = f"{filestring}_pixel{ipixel}_piximpact.fits"
    # read the piximpact file and get TIME values
    with fits.open(piximpactfile) as hdulist_piximpact:
        piximpactdata = hdulist_piximpact[1].data.copy()
    xifusimfile = f"{filestring}_pixel{ipixel}_xifusim.fits"
    if not os.path.exists(xifusimfile):
        #calculate minimum and maximum time for impacts in the pixel
        mintime = np.min(piximpactdata['TIME'])
        maxtime = np.max(piximpactdata['TIME'])
        expos_init = 1.e-6
        expos_fin = exposure
    
        #create xifusim name based on input parameters    
        comm = (f'xifusim PixImpList={piximpactfile} Streamfile={xifusimfile} '
                f'tstart={expos_init} tstop={expos_fin} '
                f'writeDRE=1 writeTrigger=0 '
                f'XMLfilename={xml_xifusim} clobber=yes ')
        
        vprint(f"  Doing simulation for pixel {ipixel} with {len(piximpactdata)} impacts ({nimpacts_inpix[ipixel]} TOTAL impacts)")
        print(f"Running {comm}")
        output_xifusim = run(comm, shell=True, capture_output=True)
        #assert output_xifusim.returncode == 0, f"xifusim failed to run: {comm}"
        assert output_xifusim.returncode == 0, print(f"xifusim failed to run: {comm} => ERROR COED: {output_xifusim.stdout.decode()}")
        assert os.path.exists(xifusimfile), f"xifusim did not produce an output file"

        # re-write correct PIXID in the xifusim file
        with fits.open(xifusimfile, mode='update') as hdulist:
            xifusimdata = hdulist[1].data
            xifusimdata['PIXID'] = ipixel
            hdulist.flush()